<a href="https://colab.research.google.com/github/FerBarrey/An-lisis-cuantitativo-avanzados/blob/main/Ejercicio_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 4


Se utiliza un modelo de Churn de usuarios de tarjetas de crédito:
https://www.kaggle.com/code/mzmdgaming/credit-card-customer-churn-model

# Cargar la base

In [1]:
import pandas as pd
import numpy as np

from google.colab import userdata
import json

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {
    'username': userdata.get('KAGGLE_USER'),
    'key': userdata.get('KAGGLE_KEY')}
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d thedevastator/predicting-credit-card-customer-attrition-with-m

Dataset URL: https://www.kaggle.com/datasets/thedevastator/predicting-credit-card-customer-attrition-with-m
License(s): CC0-1.0
100% 379k/379k [00:00<00:00, 736kB/s]
100% 379k/379k [00:00<00:00, 736kB/s]


In [2]:
# Extraigo todos los archivos que están en el .zip que acabamos de "bajar".

import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
df = pd.read_csv('/content/BankChurners.csv')

# Preguntas

Pregunta 1: ¿Cuántos de los usuarios son attrited?

Ayuda: La variable que identifica a estos usuarios es 'Attrition_Flag'

Una función posible para calcularlo es: .value_counts()

In [4]:
df['Attrition_Flag'].value_counts()

,count
Attrition_Flag,
Existing Customer,8500
Attrited Customer,1627


Pregunta 2:

El siguiente código genera una variable dummy con valor 0 para los usuarios que permanecen y valor 1 para los que se fueron.



```
df['attrited']=pd.get_dummies(df['Attrition_Flag'])['Attrited Customer']
```

Este otro genera dos datasets distintos, el primero con la información, el segundo con el objetivo



```
variables=['Customer_Age', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt']

X=df[variables]
Y=df['attrited']
```

Con esto ya podríamos correr un sencillo modelo LASSO:



```
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)
lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
lr.fit(X_train,y_train)
```

¿Cuál es el score de train y cuál el de test?

Ayuda:

```
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))
```


Ok, eso fue fácil, pero ¿Qué significan esos números?

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score


In [5]:
#El siguiente código genera una variable dummy con valor 0 para los usuarios que permanecen y valor 1 para los que se fueron.
df['attrited']=pd.get_dummies(df['Attrition_Flag'])['Attrited Customer']

In [6]:
#Este otro genera dos datasets distintos, el primero con la información, el segundo con el objetivo
variables=['Customer_Age', 'Months_on_book',
        'Total_Relationship_Count', 'Months_Inactive_12_mon',
        'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
        'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt']

X=df[variables]
Y=df['attrited']

In [7]:
#Con esto ya podríamos correr un sencillo modelo LASSO:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=1)
lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [8]:
#Score de train y  de test
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.8738427354647574
0.8790720631786771


Pregunta 3: Utilice ahora un modelo tipo elastic-net. ¿Mejora la predicción?

Ayuda:

penalty='elasticnet' y tendrá que cambiar el solver y agregar el l1_ratio

In [11]:
from sklearn.linear_model import ElasticNet
penalty='elasticnet'
solver='saga'
l1_ratio=0.5

In [14]:
#  Utilizando un  modelo tipo elastic-net para obtener el resultado de la nueva prediccion

# Elastic-net model
enet = ElasticNet(alpha=0.1, l1_ratio=0.5)
enet.fit(X_train, y_train)

# Score de train y de test para el modelo Elastic-net
print(enet.score(X_train, y_train))
print(enet.score(X_test, y_test))

0.13732524446042693
0.1571874413217098


**Respuesta 3**: La prediccion empeora usando elasticNet

Pregunta 4: ¿Cómo es la matriz de confusión? ¿Qué significa?

Ayuda:



```
from sklearn.metrics import confusion_matrix

y_pred=lr.predict(X_test)
y_true=y_test
confusion_matrix(y_true, y_pred)
```

También puede utilizar la funión .ravel() para obtener en orden los TN, FP, FN y TP.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html


Pregunta 5: ¿Cómo se puede hacer para obtener una predicción mejor que la obtenida?